In [210]:
import re

import pandas as pd
import numpy as np
import os

from numpy.ma.extras import column_stack

print(os.getcwd())

# Load dataset
df = pd.read_csv('./data/jobkorea_crawled_with_manual_v1.csv', encoding='utf-8-sig')

df['경력'] = df['경력'].astype(str).fillna("")
df['학력'] = df['학력'].astype(str).fillna("")
df['고용형태'] = df['고용형태'].astype(str).fillna("")
df['근무지'] = df['근무지'].astype(str).fillna("")
df['급여'] = df['급여'].astype(str).fillna("")
df['스킬'] = df['스킬'].astype(str).fillna("")
df['주요업무'] = df['주요업무'].astype(str).fillna("")
df['채용절차'] = df['채용절차'].astype(str).fillna("")
df['자격요건'] = df['자격요건'].astype(str).fillna("")
df['복지 및 혜택'] = df['복지 및 혜택'].astype(str).fillna("")
df['업종'] = df['업종'].astype(str).fillna("")

df.to_csv('./data/bak_jobkorea_crawled_preprocessed_3_v3.csv', index=False, encoding='utf-8-sig')

df['상세게시글id'] = df['상세게시글id'].fillna(0)
df['상세게시글id'] = df['상세게시글id'].astype(int)


C:\Users\user\PycharmProjects\portenup_ai3_project1_3\jinwoo


In [211]:
df['주요업무'].isna().sum()
df['자격요건'].isna().sum()
df['우대사항'] = df['우대사항'].fillna("")

In [212]:
import re
from kiwipiepy import Kiwi

kiwi = Kiwi()
pattern = r'[^a-zA-Z0-9가-힣]'
flags = re.compile(pattern)
compiled_work = []
compiled_requirement = []
compiled_prefer = []

for word in df['주요업무']:
    clean_word = flags.sub(' ', word)
    compiled_work.append(clean_word)

for word in df['자격요건']:
    clean_word = flags.sub(' ', word)
    compiled_requirement.append(clean_word)

for word in df['우대사항']:
    clean_word = flags.sub(' ', word)
    compiled_prefer.append(clean_word)

tokenized_work = kiwi.analyze(compiled_work)
tokenized_requirement = kiwi.analyze(compiled_requirement)
tokenized_prefer = kiwi.analyze(compiled_prefer)

none_word = ['사항','ai','경험','경험자', 'Ai', 'aI', 'AI','개발', 'to', '이상','경력','기반', '우대', '업무', '요건', '자격','활동', '관련']

work_list = []
requirement_list = []
prefer_list = []

for token in tokenized_work:
    for tlist, _ in token:
        for t in tlist:
            if t.tag not in ('NNG', 'SL') or len(t.form) < 2 or t.form in none_word:
                continue
            work_list.append(t.form)

for token in tokenized_requirement:
    for tlist, _ in token:
        for t in tlist:
            if t.tag not in ('NNG', 'SL') or len(t.form) < 2 or t.form in none_word:
                continue
            requirement_list.append(t.form)

for token in tokenized_prefer:
    for tlist, _ in token:
        for t in tlist:
            if t.tag not in ('NNG', 'SL') or len(t.form) < 2 or t.form in none_word:
                continue
            prefer_list.append(t.form)


work_frame = pd.DataFrame(work_list)
requirement_frame = pd.DataFrame(requirement_list)
prefer_frame = pd.DataFrame(prefer_list)

work_frame.value_counts()
requirement_frame.value_counts()
prefer_frame.value_counts()

0         
활용            74
설계            58
LLM           52
서비스           51
운영            49
              ..
sLLM           1
법률             1
PoC            1
Jira           1
Confluence     1
Name: count, Length: 862, dtype: int64

In [213]:
from sklearn.feature_extraction.text import CountVectorizer

In [214]:
count_vectorizer = CountVectorizer(
    max_df = 0.9,
    min_df = 1,
    max_features = 500,
    ngram_range = (1,2)
)
feat_vec = count_vectorizer.fit_transform(work_list)
feat_vec.shape

(4375, 500)

In [215]:
count_vectorizer2 = CountVectorizer(
    max_df = 0.9,
    min_df = 1,
    max_features = 500,
    ngram_range = (1,2)
)
feat_vec2 = count_vectorizer2.fit_transform(requirement_list)
feat_vec2.shape

(3351, 500)

In [216]:
count_vectorizer3 = CountVectorizer(
    max_df = 0.9,
    min_df = 1,
    max_features = 500,
    ngram_range = (1,2)
)
feat_vec3 = count_vectorizer3.fit_transform(prefer_list)
feat_vec3.shape

(3054, 500)

In [217]:
df_vec = pd.DataFrame(
    feat_vec.toarray(),
    columns = count_vectorizer.get_feature_names_out()
)
df_vec.head()

df_vec2 = pd.DataFrame(
    feat_vec2.toarray(),
    columns = count_vectorizer2.get_feature_names_out()
)
df_vec2.head()

df_vec3 = pd.DataFrame(
    feat_vec3.toarray(),
    columns = count_vectorizer3.get_feature_names_out()
)
df_vec3.head()


,account,acl,actions,adaptive,admin,ae,agent,agentic,agv,aiddison,...,혼재,혼합,홈쇼핑,확장,환경,활용,회귀,효과,효율,흡수
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [218]:
from sklearn.decomposition import LatentDirichletAllocation

In [219]:
lda = LatentDirichletAllocation(n_components=5) # 몇 개의 topic으로 할까요?
lda.fit(feat_vec)

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",5
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",10.0
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


In [220]:
from IPython.core.display import Markdown
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
display(Markdown("## LDA for JobKorea 주요업무"))
pyLDAvis.display(vis)



In [221]:
lda2 = LatentDirichletAllocation(n_components=5)  # 몇 개의 topic으로 할까요?
lda2.fit(feat_vec2)
vis2 = pyLDAvis.lda_model.prepare(lda2, feat_vec2, count_vectorizer2)
display(Markdown("## LDA for JobKorea 자격요건"))
pyLDAvis.display(vis2)



In [222]:
lda3 = LatentDirichletAllocation(n_components=5)  # 몇 개의 topic으로 할까요?
lda3.fit(feat_vec3)
vis3 = pyLDAvis.lda_model.prepare(lda3, feat_vec3, count_vectorizer3)
display(Markdown("## LDA for JobKorea 우대사항"))
pyLDAvis.display(vis3)